Found below are the libraries for generating quantile and regular forecasts in both the balancing and day ahead market.

Multi-Headed, Multi-Input Deep neural network Recurrent neural network for quantile forecast in the BM

In [ ]:

# months 1-7 search
import os ;
# os.environ['HDF5_DISABLE_VERSION_CHECK']='2'

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
import datetime as dt 
from datetime import datetime
from datetime import timedelta as td
import traceback
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
from math import floor
import seaborn as sns
from math import sqrt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from hyperopt import hp, fmin, tpe

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import *
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
import tensorflow.keras.backend as K


warnings.filterwarnings('ignore')

date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/coconnor/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)

dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()


Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:7250,:]
Y_train=Y.iloc[:7250,:]
X_test=X.iloc[7250:8739,:]
Y_test=Y.iloc[7250:8739,:]

def qloss(qs, y_true, y_pred):
    # Pinball loss for multiple quantiles
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)

loss_10th_p = lambda y_true, y_pred: qloss(0.1, y_true, y_pred)
loss_30th_p = lambda y_true, y_pred: qloss(0.3, y_true, y_pred)
loss_50th_p = lambda y_true, y_pred: qloss(0.5, y_true, y_pred)
loss_70th_p = lambda y_true, y_pred: qloss(0.7, y_true, y_pred)
loss_90th_p = lambda y_true, y_pred: qloss(0.9, y_true, y_pred)


rnn_train_LSTM_1 = X_train.loc[:,"lag_-3x1":"lag_-50x1"]
rnn_train_LSTM_2 = X_train.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_train_LSTM_3 = X_train.loc[:,"lag_-2x3":"lag_-49x3"]
rnn_train_LSTM_4 = X_train.loc[:,"lag_0x6":"lag_-47x6"]
rnn_train_LSTM_5 = X_train.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_test_LSTM_1 = X_test.loc[:,"lag_-3x1": "lag_-50x1"]
rnn_test_LSTM_2 = X_test.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_test_LSTM_3 = X_test.loc[:,"lag_-2x3": "lag_-49x3"]
rnn_test_LSTM_4 = X_test.loc[:,"lag_0x6" : "lag_-47x6"]
rnn_test_LSTM_5 = X_test.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_train_ffnn_1 = X_train.loc[:,"lag_2x7": "lag_17x7"]
rnn_train_ffnn_2 = X_train.loc[:,"lag_2x8": "lag_17x8"]
rnn_train_ffnn_3 = X_train.loc[:,"lag_2x9": "lag_17x9"]
rnn_train_ffnn_4 = X_train.loc[:,"lag_2x10": "lag_17x10"]
rnn_train_ffnn_5 = X_train.loc[:,"lag_2x11": "lag_17x11"]

rnn_test_ffnn_1 = X_test.loc[:,"lag_2x7":"lag_17x7"]
rnn_test_ffnn_2 = X_test.loc[:,"lag_2x8": "lag_17x8"]
rnn_test_ffnn_3 = X_test.loc[:,"lag_2x9": "lag_17x9"]
rnn_test_ffnn_4 = X_test.loc[:,"lag_2x10":"lag_17x10"]
rnn_test_ffnn_5 = X_test.loc[:,"lag_2x11": "lag_17x11"]

rnn_Y = Y_train.loc[:,"lag_2y": "lag_17y"]

X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

Y_train_Scaled = Y_scaler.fit_transform(Y_train)
Y_test_scaled = Y_scaler.transform(Y_test)

X_train_Scaled_LSTM = np.hstack(
    (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
     rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_train_Scaled_ffnn = np.hstack(
    (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
     rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

X_test_Scaled_LSTM = np.hstack(
    (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
     X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
     X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_test_Scaled_ffnn = np.hstack(
    (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
     X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
     X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

i_shape_lstm = (X_train_Scaled_LSTM.shape[1], X_train_Scaled_LSTM.shape[2])
i_shape_ffnn = (X_train_Scaled_ffnn.shape[1], X_train_Scaled_ffnn.shape[2])


def generate_train_and_test_dataframes(participant_df: pd.DataFrame, train_start_time: dt, train_end_time: dt, \
                                       test_start_time: dt, test_end_time: dt):

    # These are the dataframes that will be returned from the method.
    train_X_LSTM = None
    train_X_ffnn = None

    train_y = None
    test_X_LSTM = None
    test_X_ffnn = None
    test_y = None
    test_df = None
    train_df = None

    try:

        if len(participant_df) == 0:
            print("Warning: generate_train_and_test_dataframes method, participant_df has 0 rows. Ending.")
        #             return train_X, train_y, test_X, test_y, test_df

        original_columns = list(participant_df.columns)

        # Remove any rows with nan's etc (there shouldn't be any in the input).
        participant_df = participant_df.dropna()

        date_format = "%m/%d/%Y %H:%M"

        # The train dataframe, it will be used later to create train_X and train_y.
        train_df = None
        train_start_time_str = dt.datetime.strptime(train_start_time, date_format)
        train_end_time_str = dt.datetime.strptime(train_end_time, date_format)
        train_df = participant_df[
            (participant_df.index >= train_start_time_str) & (participant_df.index < train_end_time_str)].copy(
            deep="True")

        if train_df is None or len(train_df) == 0:
            print(
                "Don't have a train dataframe for train_start_time: " + train_start_time_str + ", train_end_time: " + train_end_time_str + ", exiting.")
        #             return train_X, train_y, test_X, test_y, test_df

        # Create the test dataframe, it will be used later to create test_X and test_y
        test_start_time_str = dt.datetime.strptime(test_start_time, date_format)
        test_end_time_str = dt.datetime.strptime(test_end_time, date_format)
        test_df = participant_df[
            (participant_df.index >= test_start_time_str) & (participant_df.index < test_end_time_str)].copy(
            deep="True")

        if test_df is None or len(test_df) == 0:
            print(
                "Don't have a test dataframe for test_start_time: " + test_start_time_str + ", test_end_time: " + test_end_time_str + ", exiting.")
        #             return train_X, train_y, test_X, test_y, test_df

        rnn_train_LSTM_1 = train_df.loc[:,"lag_-3x1":"lag_-50x1"]
        rnn_train_LSTM_2 = train_df.loc[:,"lag_-3x2": "lag_-50x2"]
        rnn_train_LSTM_3 = train_df.loc[:,"lag_-2x3":"lag_-49x3"]
        rnn_train_LSTM_4 = train_df.loc[:,"lag_0x6":"lag_-47x6"]
        rnn_train_LSTM_5 = train_df.loc[:,"lag_-2x12": "lag_-49x12"]

        rnn_test_LSTM_1 = test_df.loc[:,"lag_-3x1": "lag_-50x1"]
        rnn_test_LSTM_2 = test_df.loc[:,"lag_-3x2": "lag_-50x2"]
        rnn_test_LSTM_3 = test_df.loc[:,"lag_-2x3": "lag_-49x3"]
        rnn_test_LSTM_4 = test_df.loc[:,"lag_0x6" : "lag_-47x6"]
        rnn_test_LSTM_5 = test_df.loc[:,"lag_-2x12": "lag_-49x12"]
        
        rnn_train_ffnn_1 = train_df.loc[:,"lag_2x7": "lag_17x7"]
        rnn_train_ffnn_2 = train_df.loc[:,"lag_2x8": "lag_17x8"]
        rnn_train_ffnn_3 = train_df.loc[:,"lag_2x9": "lag_17x9"]
        rnn_train_ffnn_4 = train_df.loc[:,"lag_2x10": "lag_17x10"]
        rnn_train_ffnn_5 = train_df.loc[:,"lag_2x11": "lag_17x11"]
        
        rnn_test_ffnn_1 = test_df.loc[:,"lag_2x7": "lag_17x7"]
        rnn_test_ffnn_2 = test_df.loc[:,"lag_2x8": "lag_17x8"]
        rnn_test_ffnn_3 = test_df.loc[:,"lag_2x9": "lag_17x9"]
        rnn_test_ffnn_4 = test_df.loc[:,"lag_2x10": "lag_17x10"]
        rnn_test_ffnn_5 = test_df.loc[:,"lag_2x11": "lag_17x11"]

        rnn_Y = train_df.loc[:,"lag_2y": "lag_17y"]

        X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
        X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
        X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
        X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
        X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

        X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
        X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
        X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
        X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
        X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

        Y_scaler = preprocessing.MinMaxScaler()

        rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
        rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
        rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
        rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
        rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

        rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
        rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
        rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
        rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
        rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

        train_y = Y_scaler.fit_transform(rnn_Y)
        Y_scaler_n = Y_scaler.fit(rnn_Y)

        train_X_LSTM = np.hstack(
            (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
             rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
        ).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

        train_X_ffnn = np.hstack(
            (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
             rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
        ).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

        test_X_LSTM = np.hstack(
            (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
             X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
             X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
        ).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

        test_X_ffnn = np.hstack(
            (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
             X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
             X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
        ).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

        test_y = test_df.iloc[:, 0:16]

        return train_X_LSTM, train_X_ffnn, train_y, test_X_LSTM, test_X_ffnn, test_y, test_df, train_df, Y_scaler_n

    except Exception:
        print("Error: generate_train_and_test_dataframes method.")
        traceback.print_exc()
        return train_X_LSTM, train_X_ffnn, train_y, test_X_LSTM, test_X_ffnn, test_y, test_df, train_df, Y_scaler_n


def fit_multitarget_model(model, X_train_LSTM, X_train_ffnn, Y_train, X_test_LSTM, X_test_ffnn, Y_test,
                          actuals_and_forecast_df, targets, Y_scaler_n):
    """
    Fits the model to the training data.
    Then uses the test data to produce a forecast.
    Returns a dataframe containing the forecasts and actual values for each of the target variables.

    Parameters
    ----------
    model : object
        Model object i.e. randomforestregressor, linear model or other.
    X_train : dataframe
        The explanatory variables for the train/calibration set, numeric columns may already have been scaled.
    Y_train : dataframe
        The target variables for the train/calibration set. Might/Mighn't be scaled.
    X_test : dataframe
        The explanatory variables for the test set, columns may be scaled. It will comprise of 24 rows (1 row for each delivery period in the trading day).
    Y_test : dataframe
        The target variables for the test set i.e. what we would like to forecast. Similar to the previous bullet point, the dataframe will contain 24 rows.
    actuals_and_forecast_df : dataframe
        Initially the dataframe will only contain the actual values for each of the targets. At the end of the method it will also contain the forecast values.
    targets : [str]
        These are the items that we want to predict/forecast.
    scale_target_variables: boolean
       The target vector, do we want to scale it?

    Returns
    -------
    Returns a dataframe containing the forecasts and actual values for each of the target variables i.e. test set forecast and actuals.
    """
    try:
        Y_scaler = preprocessing.MinMaxScaler()
        Y_scaler = Y_scaler.fit(Y_train)
        cols = Y.columns.values.tolist()
        
        es = EarlyStopping(monitor='val_loss', mode='min',  patience=20)
        model.fit([X_train_LSTM, X_train_ffnn], Y_train, epochs=200, verbose=2,  callbacks=[es], validation_split=0.10)
        model_test_predictions = None
        model_test_predictions = pd.DataFrame(Y_scaler_n.inverse_transform(np.array(model.predict([X_test_LSTM, X_test_ffnn])).reshape(5, 16), columns=cols))
            

        print(model_test_predictions)
        print("test number of observations: " + str(len(Y_test)))

        
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_10"] = model_test_predictions.iloc[:1,i].T.tolist() if len(cols) > 1 else model_test_predictions.tolist() 
            
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_30"] = model_test_predictions.iloc[1:2,i].T.tolist() if len(cols) > 1 else model_test_predictions.tolist() 
            
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_50"] = model_test_predictions.iloc[2:3,i].T.tolist() if len(cols) > 1 else model_test_predictions.tolist()
            
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_70"] = model_test_predictions.iloc[3:4,i].T.tolist() if len(cols) > 1 else model_test_predictions.tolist() 
           
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_90"] = model_test_predictions.iloc[4:,i].T.tolist() if len(cols) > 1 else model_test_predictions.tolist()


        return actuals_and_forecast_df

    except Exception:
        print("Error: fit_multitarget_model method.")
        traceback.print_exc()
        return pd.DataFrame()


def rolling_walk_forward_validation(model, data, targets, start_time, end_time, training_days, path):
    """
    This method implements the rolling walk forward validation process.
    That is,
        (a) fit the model on the train data
        (b) use the fitted model on the test explanatory variables i.e. forecast 1 day ahead.
        (c) Move the training and test datasets forward by 1 day and repeat.
    The method will produce
        (1) A csv containing the forecast and actual target values i.e. test set output over the horizon of interest.
        (2) For each target variable, a graph of the actual and forecast values.

    Parameters
    ----------
    model: model
        The model that will be used to train the data and produce the 1 day ahead forecasts
    data: dataframe
        Participant dataframe containing the explanatory and target variables.
    explanatory_variables_of_interest : [str]
        The columns in data that will be used as explanatory variables when fitting the model.
        If there are categorical variables we want to use as explanatory variables, they are incorporated via the features_to_encode argument.
    targets: [str]
        The columns in data that we want to predict/forecast.
    features_to_encode: [str]
        If there are variables in data that we would like to apply one hot encoding to, we list them here. These one hot encoded vectors are then used as explanatory variables.
    prefix_to_include: [str]
        Just a string which will be used to name the columns if we apply one hot encoding (related to the features_to_encode argument).
     start_time: dt
       We will produce a forecast on unseen data for each trading period between [start_time, end_time].
     end_time: dt
       See previous point.
    training_days: int
       The number of training days (negative integer expected).
    path, unit_name, scenario: str
       The combination of path + unit_name + scenario indicate where the csv will be output to.
    scale_explanatory_variables: boolean
       The explanatory variables, do we want to scale them?
    scale_target_variables: boolean
       The target variables, do we want to scale them?

    Returns
    -------
    Output will include a  csv of the forecast/actual target values and a graph of the same.
    """
    try:

        all_columns = list(data.columns)
        results = pd.DataFrame()

        # Each time we
        # (a) fit the model on the calibration/train data
        # (b) apply it to the test data i.e. forecast 1 day ahead.
        # Repeat.
        date_format = "%m/%d/%Y %H:%M"
        start_time = dt.datetime.strptime(start_time, date_format)
        end_time = dt.datetime.strptime(end_time, date_format)

        while start_time < end_time:

            # Train interval
            train_start_time = start_time + td(days=training_days)
            train_end_time = start_time

            # Test interval, the test period is always the day ahead forecast
            test_start_time = train_end_time + td(hours=8)
            test_end_time = test_start_time + td(minutes=30)

            print("train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + \
                  ", test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time))

            # Generate the calibration and test dataframes.
            train_X_LSTM, train_X_ffnn, train_y, test_X_LSTM, test_X_ffnn, test_y, test_df, train_df, Y_scaler_n = generate_train_and_test_dataframes(
                participant_df=dat, train_start_time=train_start_time.strftime("%m/%d/%Y %H:%M"),
                train_end_time=train_end_time.strftime("%m/%d/%Y %H:%M"),
                test_start_time=test_start_time.strftime("%m/%d/%Y %H:%M"),
                test_end_time=test_end_time.strftime("%m/%d/%Y %H:%M"))

            if train_X_LSTM is None or len(train_X_LSTM) == 0:
                print("Don't have a train dataframe for train_start_time: " + str(
                    train_start_time) + ", train_end_time: " + str(train_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue

            if test_X_LSTM is None or len(test_X_LSTM) == 0:
                print("Don't have a test dataframe for test_start_time: " + str(
                    test_start_time) + ", test_end_time: " + str(test_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue

            # Fit the model to the train datasets, produce a forecast and return a dataframe containing the forecast/actuals.
            actuals_and_forecast_df = fit_multitarget_model(model=model, Y_scaler_n=Y_scaler_n,
                                                            X_train_LSTM=train_X_LSTM, X_train_ffnn=train_X_ffnn,
                                                            Y_train=train_y,
                                                            X_test_LSTM=test_X_LSTM, X_test_ffnn=test_X_ffnn,
                                                            Y_test=test_y,
                                                            actuals_and_forecast_df=test_df.iloc[:, 0:16],
                                                            targets=Y.columns.values.tolist())

            results = results.append(actuals_and_forecast_df)

            start_time = start_time + td(hours=8)

        results.to_csv(path + ".csv", index=False)



    except Exception:
        print("Error: rolling_walk_forward_validation method.")
        traceback.print_exc()

def create_model():    
    visible1 = Input(shape=(i_shape_lstm))
    x1=visible1
    
    for i in range(1):
        x1= LSTM(64, return_sequences= True, activation='sigmoid', input_shape=i_shape_lstm)(x1)
    x1 = Dropout(0.222222)(x1)
    
    input_1 = Flatten()(x1)   
    
    
    visible2 = Input(shape=(i_shape_ffnn))   
    x2= visible2 
    
    for i in range(2):
        x2 = Dense(64, 'tanh')(x2)
    x2 = Dropout(0.088889)(x2)
    
    input_2 = Flatten()(x2) 
            
    
    merged = concatenate([input_1, input_2])
    
    a = Dense(128, 'tanh')(merged)
    b = Dense(128, 'tanh')(merged)
    c = Dense(128, 'tanh')(merged)
    d = Dense(128, 'tanh')(merged)
    e = Dense(128, 'tanh')(merged)
        
            
    for i in range(1):
            a = Dense(128, 'relu')(a) 
    output_1 =  Dense(16, name='out_10')(a)
    
    for i in range(1):
            b = Dense(128, 'relu')(b) 
    output_2 =  Dense(16, name='out_30')(b)
    
    for i in range(1):
            c = Dense(128, 'relu')(c) 
    output_3 =  Dense(16, name='out_50')(c)
    
    for i in range(1):
            d = Dense(128, 'relu')(d) 
    output_4 =  Dense(16, name='out_70')(d)
    
    for i in range(1):
            e = Dense(128, 'relu')(e) 
    output_5 =  Dense(16, name='out_90')(e)
    
    
    model = Model(inputs=[visible1, visible2], outputs=[output_1, output_2, output_3, output_4, output_5])    
    opt = Adam(learning_rate= 0.000100)

    model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)
    return model

mmo = KerasRegressor(build_fn=create_model, epochs=200, batch_size=32, verbose=2)

rolling_walk_forward_validation(model=mmo, data=dat, start_time='6/1/2020 00:00', end_time='9/1/2020  00:00',
                                targets=Y.columns.values.tolist(), training_days=-210,
                                path="/home/coconnor/MH_Q_1-3")

Multi-Headed Deep neural network Recurrent neural network for forecasting the BM price

In [ ]:

# months 1-7 search
import os ;
# os.environ['HDF5_DISABLE_VERSION_CHECK']='2'
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from datetime import timedelta as td
import traceback
# from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso, LinearRegression
import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from math import floor
from bayes_opt import BayesianOptimization
import seaborn as sns
from math import sqrt


# tensorflow &keras
import tensorflow as tf
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import concatenate

from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
# from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)
from keras.activations import *

from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
import talos

import warnings

warnings.filterwarnings('ignore')

date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("C:/Users/ciara/OneDrive/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
# =============================================================================
# dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
dat = pd.read_csv("/home/coconnor/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)
# =============================================================================

dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()


Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:11666,:]
Y_train=Y.iloc[:11666,:]
X_test=X.iloc[11666:13155,:]
Y_test=Y.iloc[11666:13155,:]

rnn_train_LSTM_1 = X_train.loc[:,"lag_-3x1":"lag_-50x1"]
rnn_train_LSTM_2 = X_train.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_train_LSTM_3 = X_train.loc[:,"lag_-2x3":"lag_-49x3"]
rnn_train_LSTM_4 = X_train.loc[:,"lag_0x6":"lag_-47x6"]
rnn_train_LSTM_5 = X_train.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_test_LSTM_1 = X_test.loc[:,"lag_-3x1": "lag_-50x1"]
rnn_test_LSTM_2 = X_test.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_test_LSTM_3 = X_test.loc[:,"lag_-2x3": "lag_-49x3"]
rnn_test_LSTM_4 = X_test.loc[:,"lag_0x6" : "lag_-47x6"]
rnn_test_LSTM_5 = X_test.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_train_ffnn_1 = X_train.loc[:,"lag_2x7": "lag_17x7"]
rnn_train_ffnn_2 = X_train.loc[:,"lag_2x8": "lag_17x8"]
rnn_train_ffnn_3 = X_train.loc[:,"lag_2x9": "lag_17x9"]
rnn_train_ffnn_4 = X_train.loc[:,"lag_2x10": "lag_17x10"]
rnn_train_ffnn_5 = X_train.loc[:,"lag_2x11": "lag_17x11"]

rnn_test_ffnn_1 = X_test.loc[:,"lag_2x7":"lag_17x7"]
rnn_test_ffnn_2 = X_test.loc[:,"lag_2x8": "lag_17x8"]
rnn_test_ffnn_3 = X_test.loc[:,"lag_2x9": "lag_17x9"]
rnn_test_ffnn_4 = X_test.loc[:,"lag_2x10":"lag_17x10"]
rnn_test_ffnn_5 = X_test.loc[:,"lag_2x11": "lag_17x11"]

rnn_Y = Y_train.loc[:,"lag_2y": "lag_17y"]

X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

Y_train_Scaled = Y_scaler.fit_transform(Y_train)
Y_test_scaled = Y_scaler.transform(Y_test)

X_train_Scaled_LSTM = np.hstack(
    (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
     rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_train_Scaled_ffnn = np.hstack(
    (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
     rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

X_test_Scaled_LSTM = np.hstack(
    (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
     X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
     X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_test_Scaled_ffnn = np.hstack(
    (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
     X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
     X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

i_shape_lstm = (X_train_Scaled_LSTM.shape[1], X_train_Scaled_LSTM.shape[2])
i_shape_ffnn = (X_train_Scaled_ffnn.shape[1], X_train_Scaled_ffnn.shape[2])


def generate_train_and_test_dataframes(participant_df: pd.DataFrame, train_start_time: dt, train_end_time: dt, \
                                       test_start_time: dt, test_end_time: dt):


    train_X_LSTM = None
    train_X_ffnn = None

    train_y = None
    test_X_LSTM = None
    test_X_ffnn = None
    test_y = None
    test_df = None
    train_df = None

    try:

        if len(participant_df) == 0:
            print("Warning: generate_train_and_test_dataframes method, participant_df has 0 rows. Ending.")

        original_columns = list(participant_df.columns)

        participant_df = participant_df.dropna()

        date_format = "%m/%d/%Y %H:%M"

        train_df = None
        train_start_time_str = dt.datetime.strptime(train_start_time, date_format)
        train_end_time_str = dt.datetime.strptime(train_end_time, date_format)
        train_df = participant_df[
            (participant_df.index >= train_start_time_str) & (participant_df.index < train_end_time_str)].copy(
            deep="True")

        if train_df is None or len(train_df) == 0:
            print(
                "Don't have a train dataframe for train_start_time: " + train_start_time_str + ", train_end_time: " + train_end_time_str + ", exiting.")

        test_start_time_str = dt.datetime.strptime(test_start_time, date_format)
        test_end_time_str = dt.datetime.strptime(test_end_time, date_format)
        test_df = participant_df[
            (participant_df.index >= test_start_time_str) & (participant_df.index < test_end_time_str)].copy(
            deep="True")

        if test_df is None or len(test_df) == 0:
            print(
                "Don't have a test dataframe for test_start_time: " + test_start_time_str + ", test_end_time: " + test_end_time_str + ", exiting.")

        rnn_train_LSTM_1 = train_df.loc[:,"lag_-3x1":"lag_-50x1"]
        rnn_train_LSTM_2 = train_df.loc[:,"lag_-3x2": "lag_-50x2"]
        rnn_train_LSTM_3 = train_df.loc[:,"lag_-2x3":"lag_-49x3"]
        rnn_train_LSTM_4 = train_df.loc[:,"lag_0x6":"lag_-47x6"]
        rnn_train_LSTM_5 = train_df.loc[:,"lag_-2x12": "lag_-49x12"]

        rnn_test_LSTM_1 = test_df.loc[:,"lag_-3x1": "lag_-50x1"]
        rnn_test_LSTM_2 = test_df.loc[:,"lag_-3x2": "lag_-50x2"]
        rnn_test_LSTM_3 = test_df.loc[:,"lag_-2x3": "lag_-49x3"]
        rnn_test_LSTM_4 = test_df.loc[:,"lag_0x6" : "lag_-47x6"]
        rnn_test_LSTM_5 = test_df.loc[:,"lag_-2x12": "lag_-49x12"]
        
        rnn_train_ffnn_1 = train_df.loc[:,"lag_2x7": "lag_17x7"]
        rnn_train_ffnn_2 = train_df.loc[:,"lag_2x8": "lag_17x8"]
        rnn_train_ffnn_3 = train_df.loc[:,"lag_2x9": "lag_17x9"]
        rnn_train_ffnn_4 = train_df.loc[:,"lag_2x10": "lag_17x10"]
        rnn_train_ffnn_5 = train_df.loc[:,"lag_2x11": "lag_17x11"]
        
        rnn_test_ffnn_1 = test_df.loc[:,"lag_2x7": "lag_17x7"]
        rnn_test_ffnn_2 = test_df.loc[:,"lag_2x8": "lag_17x8"]
        rnn_test_ffnn_3 = test_df.loc[:,"lag_2x9": "lag_17x9"]
        rnn_test_ffnn_4 = test_df.loc[:,"lag_2x10": "lag_17x10"]
        rnn_test_ffnn_5 = test_df.loc[:,"lag_2x11": "lag_17x11"]

        rnn_Y = train_df.loc[:,"lag_2y": "lag_17y"]
        
        X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
        X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
        X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
        X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
        X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

        X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
        X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
        X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
        X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
        X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

        Y_scaler = preprocessing.MinMaxScaler()

        rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
        rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
        rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
        rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
        rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

        rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
        rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
        rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
        rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
        rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

        train_y = Y_scaler.fit_transform(rnn_Y)
        Y_scaler_n = Y_scaler.fit(rnn_Y)

        train_X_LSTM = np.hstack(
            (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
             rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
        ).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

        train_X_ffnn = np.hstack(
            (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
             rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
        ).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

        test_X_LSTM = np.hstack(
            (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
             X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
             X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
        ).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

        test_X_ffnn = np.hstack(
            (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
             X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
             X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
        ).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

        test_y = test_df.iloc[:, 0:16]

        return train_X_LSTM, train_X_ffnn, train_y, test_X_LSTM, test_X_ffnn, test_y, test_df, train_df, Y_scaler_n

    except Exception:
        print("Error: generate_train_and_test_dataframes method.")
        traceback.print_exc()
        return train_X_LSTM, train_X_ffnn, train_y, test_X_LSTM, test_X_ffnn, test_y, test_df, train_df, Y_scaler_n


def fit_multitarget_model(model, X_train_LSTM, X_train_ffnn, Y_train, X_test_LSTM, X_test_ffnn, Y_test,
                          actuals_and_forecast_df, targets, Y_scaler_n):

    try:
        Y_scaler = preprocessing.MinMaxScaler()
        Y_scaler = Y_scaler.fit(Y_train)
        cols = Y.columns.values.tolist()

        model.fit([X_train_LSTM, X_train_ffnn], Y_train)
        model_test_predictions = None
        #         model_test_predictions = model.predict(X_test)
        model_test_predictions = pd.DataFrame(
            Y_scaler_n.inverse_transform(model.predict([X_test_LSTM, X_test_ffnn]).reshape(1, 16)), columns=cols,
            index=Y_test.index)
        print(model_test_predictions)
        print("test number of observations: " + str(len(Y_test)))
        model_test_mse = mean_squared_error(Y_test, model_test_predictions)
        model_test_rmse = round(np.sqrt(model_test_mse), 2)
        model_test_mae = round(mean_absolute_error(Y_test, model_test_predictions), 2)
        print("test rmse: " + str(model_test_rmse))
        print("test mae: " + str(model_test_mae))

        for i in range(0, len(cols)):
            actuals_and_forecast_df[cols[i] + "_Forecast"] = model_test_predictions.iloc[:, i].tolist() if len(
                cols) > 1 else model_test_predictions.tolist()
            predictor_test_mse = mean_squared_error(Y_test[cols[i]], model_test_predictions.iloc[:, i]) if len(
                cols) > 1 else mean_squared_error(Y_test[cols[i]], model_test_predictions.tolist())
            predictor_test_rmse = round(np.sqrt(predictor_test_mse), 2)
            predictor_test_mae = round(mean_absolute_error(Y_test[cols[i]], model_test_predictions.iloc[:, i]),
                                       2) if len(cols) > 1 else round(
                mean_absolute_error(Y_test[cols[i]], model_test_predictions.tolist()), 2)
            print(cols[i] + " test rmse: " + str(predictor_test_rmse))
            print(cols[i] + " test mae: " + str(predictor_test_mae))

        Error_i = ([model_test_rmse, model_test_mae])
        print(Error_i)
        actuals_and_forecast_df = actuals_and_forecast_df.append(Error_i)



        return actuals_and_forecast_df

    except Exception:
        print("Error: fit_multitarget_model method.")
        traceback.print_exc()
        return pd.DataFrame()


def rolling_walk_forward_validation(model, data, targets, start_time, end_time, training_days, path):

    try:

        all_columns = list(data.columns)
        results = pd.DataFrame()


        date_format = "%m/%d/%Y %H:%M"
        start_time = dt.datetime.strptime(start_time, date_format)
        end_time = dt.datetime.strptime(end_time, date_format)

        while start_time < end_time:

            train_start_time = start_time + td(days=training_days)
            train_end_time = start_time

            test_start_time = train_end_time + td(hours=8)
            test_end_time = test_start_time + td(minutes=30)

            print("train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + \
                  ", test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time))

            train_X_LSTM, train_X_ffnn, train_y, test_X_LSTM, test_X_ffnn, test_y, test_df, train_df, Y_scaler_n = generate_train_and_test_dataframes(
                participant_df=dat, train_start_time=train_start_time.strftime("%m/%d/%Y %H:%M"),
                train_end_time=train_end_time.strftime("%m/%d/%Y %H:%M"),
                test_start_time=test_start_time.strftime("%m/%d/%Y %H:%M"),
                test_end_time=test_end_time.strftime("%m/%d/%Y %H:%M"))

            if train_X_LSTM is None or len(train_X_LSTM) == 0:
                print("Don't have a train dataframe for train_start_time: " + str(
                    train_start_time) + ", train_end_time: " + str(train_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue

            if test_X_LSTM is None or len(test_X_LSTM) == 0:
                print("Don't have a test dataframe for test_start_time: " + str(
                    test_start_time) + ", test_end_time: " + str(test_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue

            actuals_and_forecast_df = fit_multitarget_model(model=model, Y_scaler_n=Y_scaler_n,
                                                            X_train_LSTM=train_X_LSTM, X_train_ffnn=train_X_ffnn,
                                                            Y_train=train_y,
                                                            X_test_LSTM=test_X_LSTM, X_test_ffnn=test_X_ffnn,
                                                            Y_test=test_y,
                                                            actuals_and_forecast_df=test_df.iloc[:, 0:16],
                                                            targets=Y.columns.values.tolist())

            results = results.append(actuals_and_forecast_df)

            start_time = start_time + td(hours=8)

        results.to_csv(path + ".csv", index=False)



    except Exception:
        print("Error: rolling_walk_forward_validation method.")
        traceback.print_exc()

def create_model():    
            visible1 = Input(shape=(i_shape_lstm))            
            dense1 = LSTM(128, return_sequences= True, activation='tanh' , input_shape=i_shape_lstm)(visible1)
            dense2 = LSTM(128, return_sequences= True, activation='tanh' , input_shape=i_shape_lstm)(dense1)
            do_lstm = Dropout(0.044444, seed=123)(dense2)
            dense3 = LSTM(128)(do_lstm)
            flat1 = Flatten()(dense3)    
    
            visible2 = Input(shape=(i_shape_ffnn))        
            dense5 = Dense(16, activation='relu')(visible2)        
            dense6 = Dense(16, activation=LeakyReLU)(dense5)
            do_ffnn = Dropout(0.200000, seed=123)(dense6)
            dense7 = Dense(16, activation='relu')(do_ffnn)
            flat2 = Flatten()(dense7)
    
            merged = concatenate([flat1, flat2])
            dense_f = Dense(256, activation='relu')(merged)        
            outputs = Dense(16)(dense_f)

            model = Model(inputs=[visible1, visible2], outputs=outputs)    
            opt = Adam(lr=0.004522)
            model.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mean_absolute_error'])
            return model
        
es = EarlyStopping(monitor='mean_absolute_error', mode='min', verbose=0, patience=20)
mmo = KerasRegressor(build_fn=create_model, epochs=500, batch_size=48, verbose=2, callbacks=[es])

rolling_walk_forward_validation(model=mmo, data=dat, start_time='9/1/2020 00:00', end_time='12/1/2020  00:00',
                                targets=Y.columns.values.tolist(), training_days=-300,
                                path="/home/coconnor/BM_results_MH_1-13")

light gradient boosting machine for quantile forecast in the BM

In [ ]:

import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta as td
from datetime import datetime
# from pandas import Timedelta as td
import traceback
# from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/coconnor/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)


dat = dat.drop(["index"], axis=1)
dat=pd.DataFrame(dat)
dat=dat.bfill(axis ='rows')
dat=dat.ffill(axis ='rows')
dat=dat._get_numeric_data()

Y=dat.iloc[:, 0:16]

def generate_train_and_test_dataframes(participant_df: pd.DataFrame, train_start_time: dt, train_end_time: dt, \
                        test_start_time: dt, test_end_time: dt):

    train_X = None
    train_y = None 
    test_X = None 
    test_y = None 
    test_df = None

    try:
        
        if len(participant_df) == 0:
            print("Warning: generate_train_and_test_dataframes method, participant_df has 0 rows. Ending.")
            return train_X, train_y, test_X, test_y, test_df
        
        original_columns = list(participant_df.columns)

        participant_df = participant_df.dropna()

        date_format="%m/%d/%Y %H:%M"

        train_df = None
        

        
        train_start_time_str = dt.datetime.strptime(train_start_time, date_format)
        train_end_time_str = dt.datetime.strptime(train_end_time, date_format)    
        train_df = participant_df[(participant_df.index>=train_start_time_str) & (participant_df.index<train_end_time_str)].copy(deep="True")

        if train_df is None or len(train_df) == 0:
            print("Don't have a train dataframe for train_start_time: " + train_start_time_str + ", train_end_time: " + train_end_time_str + ", exiting.")            
            return train_X, train_y, test_X, test_y, test_df


        
        test_start_time_str = dt.datetime.strptime(test_start_time, date_format)    
        test_end_time_str = dt.datetime.strptime(test_end_time, date_format) 
        test_df = participant_df[(participant_df.index>=test_start_time_str) & (participant_df.index<test_end_time_str)].copy(deep="True")

        if test_df is None or len(test_df) == 0:
            print("Don't have a test dataframe for test_start_time: " + test_start_time_str + ", test_end_time: " + test_end_time_str + ", exiting.")            
            return train_X, train_y, test_X, test_y, test_df


        train_X = train_df.iloc[:, 16:]
        test_X = test_df.iloc[:, 16:]
        train_y = train_df.iloc[:, 0:16]
        test_y = test_df.iloc[:, 0:16]
                                
        return train_X, train_y, test_X, test_y, test_df
        
    except Exception:
        print("Error: generate_train_and_test_dataframes method.")
        traceback.print_exc()
        return train_X, train_y, test_X, test_y, test_df
    
    
def fit_multitarget_model(model_1, model_2, model_3, model_4, model_5, X_train, Y_train, X_test, Y_test, actuals_and_forecast_df, targets):

    try:
        
#         model.fit(X_train, Y_train) if len(targets) > 1 else model.fit(X_train, Y_train.values.ravel())  
        model_1.fit(X_train, Y_train) 
        model_2.fit(X_train, Y_train) 
        model_3.fit(X_train, Y_train) 
        model_4.fit(X_train, Y_train) 
        model_5.fit(X_train, Y_train) 


        model_test_predictions_1=None  
        model_test_predictions_3=None  
        model_test_predictions_5=None  
        model_test_predictions_7=None  
        model_test_predictions_9=None  
        model_test_predictions_1 = model_1.predict(X_test)     
        model_test_predictions_3 = model_2.predict(X_test) 
        model_test_predictions_5 = model_3.predict(X_test)     
        model_test_predictions_7 = model_4.predict(X_test)     
        model_test_predictions_9 = model_5.predict(X_test)          
                    
        cols = Y_train.columns.values.tolist()   
        
        
        print("train number of observations: " + str(len(Y_train)))

        
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_10"] = model_test_predictions_1[:,i].tolist() if len(cols) > 1 else model_test_predictions_1.tolist() 
            
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_30"] = model_test_predictions_3[:,i].tolist() if len(cols) > 1 else model_test_predictions_3.tolist() 

        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_50"] = model_test_predictions_5[:,i].tolist() if len(cols) > 1 else model_test_predictions_5.tolist() 
            
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_70"] = model_test_predictions_7[:,i].tolist() if len(cols) > 1 else model_test_predictions_7.tolist() 

        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_90"] = model_test_predictions_9[:,i].tolist() if len(cols) > 1 else model_test_predictions_9.tolist() 
          
           
        return actuals_and_forecast_df
    
    except Exception:
        print("Error: fit_multitarget_model method.")
        traceback.print_exc()
        return pd.DataFrame()
    
def rolling_walk_forward_validation(model_1, model_2, model_3, model_4, model_5, data, targets, start_time, end_time, training_days, path):

    try:

        all_columns = list(data.columns)            
        results = pd.DataFrame()
            
        #Each time we 
        # (a) fit the model on the calibration/train data
        # (b) apply it to the test data i.e. forecast 1 day ahead.
        #Repeat.
        date_format="%m/%d/%Y %H:%M"
        start_time = dt.datetime.strptime(start_time, date_format)
        end_time = dt.datetime.strptime(end_time, date_format)
        
        while start_time < end_time:
            
            #Train interval
            train_start_time = start_time + td(days=training_days)
            train_end_time = start_time 
    
            #Test interval, the test period is always the day ahead forecast
            test_start_time = train_end_time + td(hours=8)
            test_end_time = test_start_time + td(minutes=30)
            
            print("train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + \
                  ", test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time))
    
            #Generate the calibration and test dataframes.
            train_X, train_y, test_X, test_y, test_df = generate_train_and_test_dataframes(participant_df=dat, train_start_time=train_start_time.strftime("%m/%d/%Y %H:%M"), train_end_time=train_end_time.strftime("%m/%d/%Y %H:%M"), 
                            test_start_time=test_start_time.strftime("%m/%d/%Y %H:%M"), test_end_time=test_end_time.strftime("%m/%d/%Y %H:%M"))
            
            if train_X is None or len(train_X) == 0:
                print("Don't have a train dataframe for train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue
    
            if test_X is None or len(test_X) == 0:
                print("Don't have a test dataframe for test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue
            
            actuals_and_forecast_df = fit_multitarget_model(model_1=model_1,model_2=model_2,model_3=model_3, model_4=model_4,model_5=model_5, X_train=train_X, Y_train=train_y,
                                            X_test=test_X, Y_test=test_y, actuals_and_forecast_df=test_df,targets=Y.columns.values.tolist())
    
            results = results.append(actuals_and_forecast_df)
            print(results)
            start_time = start_time + td(hours=8)
            
        results.to_csv(path  + ".csv", index = False)
        
          
        
    except Exception:
        print("Error: rolling_walk_forward_validation method.")
        traceback.print_exc()
        


rolling_walk_forward_validation(model_1=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.1, learning_rate = 0.05, num_leaves=10,  max_depth = 4, n_estimators =  100)),
                                model_2=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.3, learning_rate = 0.05, num_leaves=10,  max_depth = 4, n_estimators =  100)),
                                model_3=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.5, learning_rate = 0.05, num_leaves=10,  max_depth = 4, n_estimators =  100)),
                                model_4=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.7, learning_rate = 0.05, num_leaves=10,  max_depth = 4, n_estimators =  100)),
                                model_5=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.9, learning_rate = 0.05, num_leaves=10,  max_depth = 4, n_estimators =  100)),
                                data=dat, start_time='06/1/2020 00:00',end_time='09/1/2020  00:00',
                                targets=Y.columns.values.tolist(),training_days=-210, path="/home/coconnor/lgbm_Q_1-3")

Single headed Deep neural network for quantile forecast in the DAM

In [ ]:
import os;
# os.environ['HDF5_DISABLE_VERSION_CHECK']='2'
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from datetime import timedelta as td
import traceback
# from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from math import floor
import seaborn as sns
from math import sqrt
from keras.layers import Input
from keras.models import Model

# tensorflow &keras
import tensorflow as tf
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
import tensorflow.keras.backend as K

# os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

date_format = "%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/coconnor/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)

dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')

Y = dat.iloc[:, 0:24]
X = dat.iloc[:, 24:]
X_train = X.iloc[:12360, :]
Y_train = Y.iloc[:12360, :]
X_test = X.iloc[12360:13104, :]
Y_test = Y.iloc[12360:13104, :]


def qloss(qs, y_true, y_pred):
    # Pinball loss for multiple quantiles
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q * e, (q - 1) * e)
    return K.mean(v)


loss_10th_p = lambda y_true, y_pred: qloss(0.1, y_true, y_pred)
loss_30th_p = lambda y_true, y_pred: qloss(0.3, y_true, y_pred)
loss_50th_p = lambda y_true, y_pred: qloss(0.5, y_true, y_pred)
loss_70th_p = lambda y_true, y_pred: qloss(0.7, y_true, y_pred)
loss_90th_p = lambda y_true, y_pred: qloss(0.9, y_true, y_pred)



rnn_train_1 = X_train.loc[:, "EURPrices-24":"EURPrices-167"]
rnn_train_2 = X_train.loc[:, "WF": "WF-143"]
rnn_train_3 = X_train.loc[:, "DF": "DF-143"]

rnn_test_1 = X_test.loc[:, "EURPrices-24":"EURPrices-167"]
rnn_test_2 = X_test.loc[:, "WF": "WF-143"]
rnn_test_3 = X_test.loc[:, "DF": "DF-143"]

rnn_Y = Y_train.loc[:, "EURPrices":"EURPrices+23"]
rnn_test_Y = Y_test.loc[:, "EURPrices":"EURPrices+23"]

X_scaler1 = preprocessing.MinMaxScaler()
X_scaler2 = preprocessing.MinMaxScaler()
X_scaler3 = preprocessing.MinMaxScaler()
Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train_1 = X_scaler1.fit_transform(rnn_train_1)
rnn_scaled_train_2 = X_scaler2.fit_transform(rnn_train_2)
rnn_scaled_train_3 = X_scaler3.fit_transform(rnn_train_3)

Y_train_Scaled = Y_scaler.fit_transform(rnn_Y)
Y_test_scaled = Y_scaler.transform(rnn_test_Y)

X_train_Scaled = np.hstack(
    (rnn_scaled_train_1, rnn_scaled_train_2, rnn_scaled_train_3)
).reshape(rnn_train_1.shape[0], 3, 144).transpose(0, 2, 1)

X_test_Scaled = np.hstack(
    (X_scaler1.transform(rnn_test_1), X_scaler2.transform(rnn_test_2), X_scaler3.transform(rnn_test_3))
).reshape(rnn_test_1.shape[0], 3, 144).transpose(0, 2, 1)

i_shape = (X_train_Scaled.shape[1], X_train_Scaled.shape[2])



def generate_train_and_test_dataframes(participant_df: pd.DataFrame, train_start_time: dt, train_end_time: dt, \
                                       test_start_time: dt, test_end_time: dt):


    # These are the dataframes that will be returned from the method.
    train_X = None
    train_y = None
    test_X = None
    test_y = None
    test_df = None
    train_df = None

    try:

        if len(participant_df) == 0:
            print("Warning: generate_train_and_test_dataframes method, participant_df has 0 rows. Ending.")
        #             return train_X, train_y, test_X, test_y, test_df

        original_columns = list(participant_df.columns)

        # Remove any rows with nan's etc (there shouldn't be any in the input).
        participant_df = participant_df.dropna()

        date_format = "%d/%m/%Y %H:%M"

        # The train dataframe, it will be used later to create train_X and train_y.
        train_df = None
        train_start_time_str = dt.datetime.strptime(train_start_time, date_format)
        train_end_time_str = dt.datetime.strptime(train_end_time, date_format)
        train_df = participant_df[
            (participant_df.index >= train_start_time_str) & (participant_df.index < train_end_time_str)].copy(
            deep="True")

        if train_df is None or len(train_df) == 0:
            print(
                "Don't have a train dataframe for train_start_time: " + train_start_time_str + ", train_end_time: " + train_end_time_str + ", exiting.")
        #             return train_X, train_y, test_X, test_y, test_df

        # Create the test dataframe, it will be used later to create test_X and test_y
        test_start_time_str = dt.datetime.strptime(test_start_time, date_format)
        test_end_time_str = dt.datetime.strptime(test_end_time, date_format)
        test_df = participant_df[
            (participant_df.index >= test_start_time_str) & (participant_df.index < test_end_time_str)].copy(
            deep="True")

        if test_df is None or len(test_df) == 0:
            print(
                "Don't have a test dataframe for test_start_time: " + test_start_time_str + ", test_end_time: " + test_end_time_str + ", exiting.")
        #             return train_X, train_y, test_X, test_y, test_df

        rnn_train_1 = train_df.loc[:, "EURPrices-24":"EURPrices-167"]
        rnn_train_2 = train_df.loc[:, "WF": "WF-143"]
        rnn_train_3 = train_df.loc[:, "DF": "DF-143"]

        rnn_test_1 = test_df.loc[:, "EURPrices-24":"EURPrices-167"]
        rnn_test_2 = test_df.loc[:, "WF": "WF-143"]
        rnn_test_3 = test_df.loc[:, "DF": "DF-143"]

        rnn_Y = train_df.loc[:, "EURPrices":"EURPrices+23"]
        rnn_test_Y = test_df.loc[:, "EURPrices":"EURPrices+23"]

        X_scaler1 = preprocessing.MinMaxScaler()
        X_scaler2 = preprocessing.MinMaxScaler()
        X_scaler3 = preprocessing.MinMaxScaler()
        Y_scaler = preprocessing.MinMaxScaler()

        rnn_scaled_train_1 = X_scaler1.fit_transform(rnn_train_1)
        rnn_scaled_train_2 = X_scaler2.fit_transform(rnn_train_2)
        rnn_scaled_train_3 = X_scaler3.fit_transform(rnn_train_3)

        train_y = Y_scaler.fit_transform(rnn_Y)
        Y_scaler_n = Y_scaler.fit(rnn_Y)

        Y_test_scaled = Y_scaler.transform(rnn_test_Y)

        train_X = np.hstack(
            (rnn_scaled_train_1, rnn_scaled_train_2, rnn_scaled_train_3)
        ).reshape(rnn_train_1.shape[0], 3, 144).transpose(0, 2, 1)

        test_X = np.hstack(
            (X_scaler1.transform(rnn_test_1), X_scaler2.transform(rnn_test_2), X_scaler3.transform(rnn_test_3))
        ).reshape(rnn_test_1.shape[0], 3, 144).transpose(0, 2, 1)

        test_y = test_df.iloc[:, 0:24]

        return train_X, train_y, test_X, test_y, test_df, train_df, Y_scaler_n

    except Exception:
        print("Error: generate_train_and_test_dataframes method.")
        traceback.print_exc()
        return train_X, train_y, test_X, test_y, test_df, train_df, Y_scaler_n


def fit_multitarget_model(model, X_train, Y_train, X_test, Y_test, actuals_and_forecast_df, targets, Y_scaler_n):
    try:
        Y_scaler = preprocessing.MinMaxScaler()
        Y_scaler = Y_scaler.fit(Y_train)
        cols = Y.columns.values.tolist()
        es = EarlyStopping(monitor='val_loss', mode='min',  patience=20)

        model.fit(X_train, Y_train, epochs=200, verbose=2,  callbacks=[es], validation_split=0.1)
        model_test_predictions=None
        model_test_predictions = pd.DataFrame(Y_scaler_n.inverse_transform(np.array(model.predict(X_test).reshape(5,24))), columns=cols)

        print(model_test_predictions)
        print("test number of observations: " + str(len(Y_test)))

        for i in range(0, len(cols)):
            actuals_and_forecast_df[cols[i] + "_Forecast_10"] = model_test_predictions.iloc[:1, i].T.tolist() if len(
                cols) > 1 else model_test_predictions.tolist()

        for i in range(0, len(cols)):
            actuals_and_forecast_df[cols[i] + "_Forecast_30"] = model_test_predictions.iloc[1:2, i].T.tolist() if len(
                cols) > 1 else model_test_predictions.tolist()

        for i in range(0, len(cols)):
            actuals_and_forecast_df[cols[i] + "_Forecast_50"] = model_test_predictions.iloc[2:3, i].T.tolist() if len(
                cols) > 1 else model_test_predictions.tolist()

        for i in range(0, len(cols)):
            actuals_and_forecast_df[cols[i] + "_Forecast_70"] = model_test_predictions.iloc[3:4, i].T.tolist() if len(
                cols) > 1 else model_test_predictions.tolist()

        for i in range(0, len(cols)):
            actuals_and_forecast_df[cols[i] + "_Forecast_90"] = model_test_predictions.iloc[4:, i].T.tolist() if len(
                cols) > 1 else model_test_predictions.tolist()

        return actuals_and_forecast_df

    except Exception:
        print("Error: fit_multitarget_model method.")
        traceback.print_exc()
        return pd.DataFrame()


def rolling_walk_forward_validation(model, data, targets, start_time, end_time, training_days, path):
    try:

        all_columns = list(data.columns)
        results = pd.DataFrame()

        date_format = "%d/%m/%Y %H:%M"
        start_time = dt.datetime.strptime(start_time, date_format)
        end_time = dt.datetime.strptime(end_time, date_format)

        while start_time < end_time:

            # Train interval
            train_start_time = start_time + td(days=training_days)
            train_end_time = start_time

            # Test interval, the test period is always the day ahead forecast
            test_start_time = train_end_time + td(hours=0)
            test_end_time = test_start_time + td(hours=1)

            print("train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + \
                  ", test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time))

            # Generate the calibration and test dataframes.
            train_X, train_y, test_X, test_y, test_df, train_df, Y_scaler_n = generate_train_and_test_dataframes(
                participant_df=dat, train_start_time=train_start_time.strftime("%d/%m/%Y %H:%M"),
                train_end_time=train_end_time.strftime("%d/%m/%Y %H:%M"),
                test_start_time=test_start_time.strftime("%d/%m/%Y %H:%M"),
                test_end_time=test_end_time.strftime("%d/%m/%Y %H:%M"))

            if train_X is None or len(train_X) == 0:
                print("Don't have a train dataframe for train_start_time: " + str(
                    train_start_time) + ", train_end_time: " + str(train_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue

            if test_X is None or len(test_X) == 0:
                print("Don't have a test dataframe for test_start_time: " + str(
                    test_start_time) + ", test_end_time: " + str(test_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue

            # Fit the model to the train datasets, produce a forecast and return a dataframe containing the forecast/actuals.
            actuals_and_forecast_df = fit_multitarget_model(model=model, Y_scaler_n=Y_scaler_n, X_train=train_X,
                                                            Y_train=train_y,
                                                            X_test=test_X, Y_test=test_y,
                                                            actuals_and_forecast_df=test_df.iloc[:, 0:24],
                                                            targets=Y.columns.values.tolist())

            results = results.append(actuals_and_forecast_df)
            start_time = start_time + td(hours=24)

        results.to_csv(path + ".csv", index=False)



    except Exception:
        print("Error: rolling_walk_forward_validation method.")
        traceback.print_exc()


def create_model():
    net_input = Input(shape=i_shape)

    x = Flatten(input_shape=i_shape)(net_input)

    for i in range(3):
        x = Dense(144, 'sigmoid')(x)

    for i in range(3):
        x = Dense(240, 'tanh')(x)
    x = Dropout(0.311111)(x)

    a = Dense(24, 'tanh')(x)
    b = Dense(24, 'tanh')(x)
    c = Dense(24, 'tanh')(x)
    d = Dense(24, 'tanh')(x)
    e = Dense(24, 'tanh')(x)

    for i in range(1):
        a = Dense(48, 'relu')(a)
    output_1 = Dense(24, name='out_10')(a)

    for i in range(1):
        b = Dense(48, 'relu')(b)
    output_2 = Dense(24, name='out_30')(b)

    for i in range(1):
        c = Dense(48, 'relu')(c)
    output_3 = Dense(24, name='out_50')(c)

    for i in range(1):
        d = Dense(48, 'relu')(d)
    output_4 = Dense(24, name='out_70')(d)

    for i in range(1):
        e = Dense(48, 'relu')(e)
    output_5 = Dense(24, name='out_90')(e)

    opt = Adam(learning_rate=0.000100)
    model = Model(inputs=net_input, outputs=[output_1, output_2, output_3, output_4, output_5])
    model.compile(loss=[loss_10th_p, loss_30th_p, loss_50th_p, loss_70th_p, loss_90th_p], optimizer=opt)

    return model


mmo = KerasRegressor(build_fn=create_model, epochs=200, batch_size=16, verbose=2)

rolling_walk_forward_validation(model=mmo, data=dat, start_time='1/6/2020 00:00', end_time='1/9/2020  00:00',
                                targets=Y.columns.values.tolist(), training_days=-210,
                                path="/home/coconnor/SH_Q_DAM_1-3")

light gradient boosting library for quantile forecast in the DAM 

In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta as td
from datetime import datetime
# from pandas import Timedelta as td
import traceback
# from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

date_format = "%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/coconnor/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)

dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')

Y = dat.iloc[:, 0:24]


def generate_train_and_test_dataframes(participant_df: pd.DataFrame, train_start_time: dt, train_end_time: dt, \
                        test_start_time: dt, test_end_time: dt):

    train_X = None
    train_y = None 
    test_X = None 
    test_y = None 
    test_df = None

    try:
        
        if len(participant_df) == 0:
            print("Warning: generate_train_and_test_dataframes method, participant_df has 0 rows. Ending.")
            return train_X, train_y, test_X, test_y, test_df
        
        original_columns = list(participant_df.columns)

        participant_df = participant_df.dropna()

        date_format = "%d/%m/%Y %H:%M"

        train_df = None
        

        
        train_start_time_str = dt.datetime.strptime(train_start_time, date_format)
        train_end_time_str = dt.datetime.strptime(train_end_time, date_format)    
        train_df = participant_df[(participant_df.index>=train_start_time_str) & (participant_df.index<train_end_time_str)].copy(deep="True")

        if train_df is None or len(train_df) == 0:
            print("Don't have a train dataframe for train_start_time: " + train_start_time_str + ", train_end_time: " + train_end_time_str + ", exiting.")            
            return train_X, train_y, test_X, test_y, test_df


        
        test_start_time_str = dt.datetime.strptime(test_start_time, date_format)    
        test_end_time_str = dt.datetime.strptime(test_end_time, date_format) 
        test_df = participant_df[(participant_df.index>=test_start_time_str) & (participant_df.index<test_end_time_str)].copy(deep="True")

        if test_df is None or len(test_df) == 0:
            print("Don't have a test dataframe for test_start_time: " + test_start_time_str + ", test_end_time: " + test_end_time_str + ", exiting.")            
            return train_X, train_y, test_X, test_y, test_df


        train_X = train_df.iloc[:, 24:]
        test_X = test_df.iloc[:, 24:]
        train_y = train_df.iloc[:, 0:24]
        test_y = test_df.iloc[:, 0:24]
                                
        return train_X, train_y, test_X, test_y, test_df
        
    except Exception:
        print("Error: generate_train_and_test_dataframes method.")
        traceback.print_exc()
        return train_X, train_y, test_X, test_y, test_df
    
    
def fit_multitarget_model(model_1, model_2, model_3, model_4, model_5, X_train, Y_train, X_test, Y_test, actuals_and_forecast_df, targets):

    try:
        
#         model.fit(X_train, Y_train) if len(targets) > 1 else model.fit(X_train, Y_train.values.ravel())  
        model_1.fit(X_train, Y_train) 
        model_2.fit(X_train, Y_train) 
        model_3.fit(X_train, Y_train) 
        model_4.fit(X_train, Y_train) 
        model_5.fit(X_train, Y_train) 


        model_test_predictions_1=None  
        model_test_predictions_3=None  
        model_test_predictions_5=None  
        model_test_predictions_7=None  
        model_test_predictions_9=None  
        model_test_predictions_1 = model_1.predict(X_test)     
        model_test_predictions_3 = model_2.predict(X_test) 
        model_test_predictions_5 = model_3.predict(X_test)     
        model_test_predictions_7 = model_4.predict(X_test)     
        model_test_predictions_9 = model_5.predict(X_test)          
                    
        cols = Y_train.columns.values.tolist()   
        
        
        print("train number of observations: " + str(len(Y_train)))

        
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_10"] = model_test_predictions_1[:,i].tolist() if len(cols) > 1 else model_test_predictions_1.tolist() 
            
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_30"] = model_test_predictions_3[:,i].tolist() if len(cols) > 1 else model_test_predictions_3.tolist() 

        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_50"] = model_test_predictions_5[:,i].tolist() if len(cols) > 1 else model_test_predictions_5.tolist() 
            
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_70"] = model_test_predictions_7[:,i].tolist() if len(cols) > 1 else model_test_predictions_7.tolist() 

        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_90"] = model_test_predictions_9[:,i].tolist() if len(cols) > 1 else model_test_predictions_9.tolist() 
          
           
        return actuals_and_forecast_df
    
    except Exception:
        print("Error: fit_multitarget_model method.")
        traceback.print_exc()
        return pd.DataFrame()
    
def rolling_walk_forward_validation(model_1, model_2, model_3, model_4, model_5, data, targets, start_time, end_time, training_days, path):

    try:

        all_columns = list(data.columns)            
        results = pd.DataFrame()
            

        date_format = "%d/%m/%Y %H:%M"
        start_time = dt.datetime.strptime(start_time, date_format)
        end_time = dt.datetime.strptime(end_time, date_format)
        
        while start_time < end_time:
            
            #Train interval
            train_start_time = start_time + td(days=training_days)
            train_end_time = start_time 
    
            #Test interval, the test period is always the day ahead forecast
            test_start_time = train_end_time + td(hours=0)
            test_end_time = test_start_time + td(hours=1)
            
            print("train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + \
                  ", test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time))
    
            #Generate the calibration and test dataframes.
            train_X, train_y, test_X, test_y, test_df = generate_train_and_test_dataframes(participant_df=dat, train_start_time=train_start_time.strftime("%d/%m/%Y %H:%M"), train_end_time=train_end_time.strftime("%d/%m/%Y %H:%M"), 
                            test_start_time=test_start_time.strftime("%d/%m/%Y %H:%M"), test_end_time=test_end_time.strftime("%d/%m/%Y %H:%M"))
            
            if train_X is None or len(train_X) == 0:
                print("Don't have a train dataframe for train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue
    
            if test_X is None or len(test_X) == 0:
                print("Don't have a test dataframe for test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue
            
            actuals_and_forecast_df = fit_multitarget_model(model_1=model_1,model_2=model_2,model_3=model_3, model_4=model_4,model_5=model_5, X_train=train_X, Y_train=train_y,
                                            X_test=test_X, Y_test=test_y, actuals_and_forecast_df=test_df,targets=Y.columns.values.tolist())
    
            results = results.append(actuals_and_forecast_df)
            print(results)
            start_time = start_time + td(hours=24)
            
        results.to_csv(path  + ".csv", index = False)
        
          
        
    except Exception:
        print("Error: rolling_walk_forward_validation method.")
        traceback.print_exc()
        


rolling_walk_forward_validation(model_1=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.1, learning_rate = 0.01, num_leaves=20,  max_depth = 18, n_estimators =  60000)),
                                model_2=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.3, learning_rate = 0.01, num_leaves=20,  max_depth = 18, n_estimators =  60000)),
                                model_3=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.5, learning_rate = 0.01, num_leaves=20,  max_depth = 18, n_estimators =  60000)),
                                model_4=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.7, learning_rate = 0.01, num_leaves=20,  max_depth = 18, n_estimators =  60000)),
                                model_5=MultiOutputRegressor(lgb.LGBMRegressor(objective = 'quantile', alpha=0.9, learning_rate = 0.01, num_leaves=20,  max_depth = 18, n_estimators =  60000)),
                                data=dat, start_time='1/6/2020 00:00', end_time='1/9/2020  00:00',
                                targets=Y.columns.values.tolist(),training_days=-210, path="/home/coconnor/lgbm_Q_DAM_1-3")


random forest & KNN library for quantile forecast in the DAM 

In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta as td
from datetime import datetime
# from pandas import Timedelta as td
import traceback
# from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn_quantile import RandomForestQuantileRegressor


import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

date_format = "%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/coconnor/DAM_VAR_1-3.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)

dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')

Y = dat.iloc[:, 0:24]

def generate_train_and_test_dataframes(participant_df: pd.DataFrame, train_start_time: dt, train_end_time: dt, \
                        test_start_time: dt, test_end_time: dt):

    train_X = None
    train_y = None 
    test_X = None 
    test_y = None 
    test_df = None

    try:
        
        if len(participant_df) == 0:
            print("Warning: generate_train_and_test_dataframes method, participant_df has 0 rows. Ending.")
            return train_X, train_y, test_X, test_y, test_df
        
        original_columns = list(participant_df.columns)

        participant_df = participant_df.dropna()

        date_format = "%d/%m/%Y %H:%M"

        train_df = None
        

        
        train_start_time_str = dt.datetime.strptime(train_start_time, date_format)
        train_end_time_str = dt.datetime.strptime(train_end_time, date_format)    
        train_df = participant_df[(participant_df.index>=train_start_time_str) & (participant_df.index<train_end_time_str)].copy(deep="True")

        if train_df is None or len(train_df) == 0:
            print("Don't have a train dataframe for train_start_time: " + train_start_time_str + ", train_end_time: " + train_end_time_str + ", exiting.")            
            return train_X, train_y, test_X, test_y, test_df


        
        test_start_time_str = dt.datetime.strptime(test_start_time, date_format)    
        test_end_time_str = dt.datetime.strptime(test_end_time, date_format) 
        test_df = participant_df[(participant_df.index>=test_start_time_str) & (participant_df.index<test_end_time_str)].copy(deep="True")

        if test_df is None or len(test_df) == 0:
            print("Don't have a test dataframe for test_start_time: " + test_start_time_str + ", test_end_time: " + test_end_time_str + ", exiting.")            
            return train_X, train_y, test_X, test_y, test_df


        train_X = train_df.iloc[:, 24:]
        test_X = test_df.iloc[:, 24:]
        train_y = train_df.iloc[:, 0:24]
        test_y = test_df.iloc[:, 0:24]
                                
        return train_X, train_y, test_X, test_y, test_df
        
    except Exception:
        print("Error: generate_train_and_test_dataframes method.")
        traceback.print_exc()
        return train_X, train_y, test_X, test_y, test_df
    
    
def fit_multitarget_model(model, X_train, Y_train, X_test, Y_test, actuals_and_forecast_df, targets):
  
    try:
        
        model.fit(X_train, Y_train) 

        model_test_predictions=None  
        model_test_predictions = model.predict(X_test).reshape(5,24)                            
        cols = Y_train.columns.values.tolist()   
        
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_10"] = model_test_predictions[0:1,i].tolist() if len(cols) > 1 else model_test_predictions.tolist() 
            
        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_30"] = model_test_predictions[1:2,i].tolist() if len(cols) > 1 else model_test_predictions.tolist() 

        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_50"] = model_test_predictions[2:3,i].tolist() if len(cols) > 1 else model_test_predictions.tolist() 

        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_70"] = model_test_predictions[3:4,i].tolist() if len(cols) > 1 else model_test_predictions.tolist() 

        for i in range(0,len(cols)):    
            actuals_and_forecast_df[cols[i]+"_Forecast_90"] = model_test_predictions[4:,i].tolist() if len(cols) > 1 else model_test_predictions.tolist()
            
        print("train number of observations: " + str(len(Y_train)))
        
        
        
           
        return actuals_and_forecast_df
    
    except Exception:
        print("Error: fit_multitarget_model method.")
        traceback.print_exc()
        return pd.DataFrame()
    
def rolling_walk_forward_validation(model, data, targets, start_time, end_time, training_days, path):
 
    try:

        all_columns = list(data.columns)            
        results = pd.DataFrame()
            

        date_format="%d/%m/%Y %H:%M"
        start_time = dt.datetime.strptime(start_time, date_format)
        end_time = dt.datetime.strptime(end_time, date_format)
        
        while start_time < end_time:
            
            train_start_time = start_time + td(days=training_days)
            train_end_time = start_time 
    
            test_start_time = train_end_time + td(hours=0)
            test_end_time = test_start_time + td(hours=1)
            
            print("train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + \
                  ", test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time))
    
            train_X, train_y, test_X, test_y, test_df = generate_train_and_test_dataframes(participant_df=dat, train_start_time=train_start_time.strftime("%d/%m/%Y %H:%M"), train_end_time=train_end_time.strftime("%d/%m/%Y %H:%M"), 
                            test_start_time=test_start_time.strftime("%d/%m/%Y %H:%M"), test_end_time=test_end_time.strftime("%d/%m/%Y %H:%M"))
            
            if train_X is None or len(train_X) == 0:
                print("Don't have a train dataframe for train_start_time: " + str(train_start_time) + ", train_end_time: " + str(train_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue
    
            if test_X is None or len(test_X) == 0:
                print("Don't have a test dataframe for test_start_time: " + str(test_start_time) + ", test_end_time: " + str(test_end_time) + ", skipping.")
                start_time = start_time + td(days=training_days)
                continue
            
            actuals_and_forecast_df = fit_multitarget_model(model=model, X_train=train_X, Y_train=train_y,
                                            X_test=test_X, Y_test=test_y, actuals_and_forecast_df=test_df,targets=Y.columns.values.tolist())
    
            results = results.append(actuals_and_forecast_df)
            print(results)
            start_time = start_time + td(hours=24)
            
        results.to_csv(path  + ".csv", index = False)
        
          
        
    except Exception:
        print("Error: rolling_walk_forward_validation method.")
        traceback.print_exc()


rolling_walk_forward_validation(model = MultiOutputRegressor(RandomForestQuantileRegressor(q=[0.10, 0.30, 0.50, 0.70, 0.90], max_depth=2 , min_samples_leaf=2 , n_estimators=100 , min_samples_split=2 )),
                                data=dat, start_time='1/6/2020 00:00', end_time='1/9/2020  00:00',
                                targets=Y.columns.values.tolist(),training_days=-210, path="/home/coconnor/rf_Q_DAM_1-3")